In [13]:
import sys
sys.path.append("../../")

In [14]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from itops.config.configs import CONFIGS

True


In [2]:
df_themes = pd.read_csv("ITSM_Data_THEMES_ONLY.csv")

In [3]:
from sentence_transformers import SentenceTransformer
MODEL_NAME = "all-MiniLM-L6-v2"

c:\Ambarish\itops\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
def get_embedding_query_vector(query,model_name):
        """Get the vector of the query

        Args:
            query (string): user input

        Returns:
            _type_: vector of the query
        """
        model = SentenceTransformer(model_name)
        query_vector = model.encode(query)
        return query_vector

In [5]:
len(df_themes)

118

In [6]:
embedding_list = []
for i in range(len(df_themes)):
    content = df_themes.iloc[i]["themes"]
    embedding = get_embedding_query_vector(content,MODEL_NAME)
    embedding_list.append(embedding)
    print(f"Completed EMBEDDING  {i+1} ROW")

Completed EMBEDDING  1 ROW
Completed EMBEDDING  2 ROW
Completed EMBEDDING  3 ROW
Completed EMBEDDING  4 ROW
Completed EMBEDDING  5 ROW
Completed EMBEDDING  6 ROW
Completed EMBEDDING  7 ROW
Completed EMBEDDING  8 ROW
Completed EMBEDDING  9 ROW
Completed EMBEDDING  10 ROW
Completed EMBEDDING  11 ROW
Completed EMBEDDING  12 ROW
Completed EMBEDDING  13 ROW
Completed EMBEDDING  14 ROW
Completed EMBEDDING  15 ROW
Completed EMBEDDING  16 ROW
Completed EMBEDDING  17 ROW
Completed EMBEDDING  18 ROW
Completed EMBEDDING  19 ROW
Completed EMBEDDING  20 ROW
Completed EMBEDDING  21 ROW
Completed EMBEDDING  22 ROW
Completed EMBEDDING  23 ROW
Completed EMBEDDING  24 ROW
Completed EMBEDDING  25 ROW
Completed EMBEDDING  26 ROW
Completed EMBEDDING  27 ROW
Completed EMBEDDING  28 ROW
Completed EMBEDDING  29 ROW
Completed EMBEDDING  30 ROW
Completed EMBEDDING  31 ROW
Completed EMBEDDING  32 ROW
Completed EMBEDDING  33 ROW
Completed EMBEDDING  34 ROW
Completed EMBEDDING  35 ROW
Completed EMBEDDING  36 ROW
C

In [7]:
type(embedding_list)

list

In [8]:
df_themes["embeddings"] = embedding_list

In [9]:
kmeans = KMeans(n_clusters=5, random_state=0)
embedding_array = np.array(df_themes["embeddings"].tolist())

kmeans.fit(embedding_array)

KMeans(n_clusters=5, random_state=0)

In [10]:
labels = kmeans.labels_
df_themes["CLUSTERS"] = labels
df_themes.to_csv("CLUSTERS001.csv",index = False)

In [17]:
num_clusters = 5
df = df_themes
user_input = "Please extract the theme of the provided context in Maximum 5 words"


In [18]:
from itops.llm.azureopenaimanager.azure_open_ai_helper import AzureOpenAIManager
azure_open_ai_helper = AzureOpenAIManager(endpoint=CONFIGS.AZURE_OPENAI_ENDPOINT,
                                          api_key =CONFIGS.AZURE_OPENAI_API_KEY,
                                          deployment_id=CONFIGS.AZURE_OPENAI_DEPLOYMENT_ID,
                    api_version="2023-05-15")

In [19]:
cluster_name_list = []
for i in range(num_clusters):
    df_cluster = df[df["CLUSTERS"] == i]
    full_content = ""
    for j in range(len(df_cluster)):
        content = df_cluster.iloc[j]["Text"]
        full_content = full_content + str(content) + " \n "
    
    cluster_name = azure_open_ai_helper.generate_reply_from_context(user_input, 
                                                            full_content, 
                                                            conversation=[])
    cluster_name_list.append(cluster_name[0])
    cluster_name =""
    print(f"Completed CLUSTERING PART 1 {i+1} ROW")


Completed CLUSTERING PART 1 1 ROW
Completed CLUSTERING PART 1 2 ROW
Completed CLUSTERING PART 1 3 ROW
Completed CLUSTERING PART 1 4 ROW
Completed CLUSTERING PART 1 5 ROW


In [20]:
cluster_name_list

['IT Support and Access Issues',
 'Outlook Email Issues and Errors',
 'Technical Issues with Internet Browsers',
 'Device replacement and ordering accessories',
 'Technical Support for Software Issues']

In [21]:
cluster_number_list= []
for i in range(len(cluster_name_list)):
    cluster_number_list.append(i)
df_clusters = pd.DataFrame()
df_clusters["CLUSTERS"] = cluster_number_list
df_clusters["CLUSTER_NAMES"] = cluster_name_list

In [22]:
df_clusters

,CLUSTERS,CLUSTER_NAMES
0,0,IT Support and Access Issues
1,1,Outlook Email Issues and Errors
2,2,Technical Issues with Internet Browsers
3,3,Device replacement and ordering accessories
4,4,Technical Support for Software Issues


In [23]:
df.columns

Index(['ID_request', 'Text', 'Category', 'Solution', 'Date_request_recieved',
       'Date_request_solved', 'ID_agent', 'themes', 'embeddings', 'CLUSTERS'],
      dtype='object')

In [24]:
df_all = df.merge(df_clusters)
print(f"Completed CLUSTERING FINAL")

Completed CLUSTERING FINAL


In [25]:
df_all

,ID_request,Text,Category,Solution,Date_request_recieved,Date_request_solved,ID_agent,themes,embeddings,CLUSTERS,CLUSTER_NAMES
0,2913,Discord issue,Discord,Problem solved,03-10-2022,03-10-2022,3,Discord server management and moderation.,"[0.03435141, -0.035914615, 0.05115362, 0.04248...",4,Technical Support for Software Issues
1,2691,I need to install Discord on my new PC.,Discord,Discord Installed,08-10-2022,10-10-2022,1,Installing Discord on new PC.,"[0.05971722, -0.101868406, 0.071770936, -0.082...",4,Technical Support for Software Issues
2,3395,Discord disappeared from laptop after I update...,Discord,Discord Installed,17-10-2022,17-10-2022,3,Discord disappeared after Windows update.,"[0.01854497, -0.033583798, 0.05134876, 0.08840...",4,Technical Support for Software Issues
3,2380,Discord is very slow while other applications ...,Discord,"Software updated, PC cleaned.",11-10-2022,11-10-2022,2,Technical issue with Discord.,"[0.035404805, -0.05236258, 0.062468614, 0.0057...",4,Technical Support for Software Issues
4,4999,Discor needed for instalation.,Discord,Problem solved,04-10-2022,04-10-2022,2,Discord installation requirements.,"[0.07610816, -0.119744405, 0.08222655, -0.0545...",4,Technical Support for Software Issues
...,...,...,...,...,...,...,...,...,...,...,...
113,2455,Ich möchte eine zusätzliche Tastatur bestellen...,Keyboard,Item sent,06-10-2022,06-10-2022,3,Ordering additional keyboard for office,"[-0.016174713, -0.06935228, 0.084742524, -0.03...",3,Device replacement and ordering accessories
114,3853,Der Tastaturaustausch ist erforderlich. Bitte ...,Keyboard,Item sent,17-10-2022,17-10-2022,3,Keyboard replacement required.,"[-0.04847761, -0.059213486, 0.07759717, -0.078...",3,Device replacement and ordering accessories
115,4541,I would like to order the new keyboard for pre...,Keyboard,Item sent,19-10-2022,20-10-2022,2,Ordering new keyboard for presentation.,"[-0.010398116, -0.07294827, 0.058754314, -0.06...",3,Device replacement and ordering accessories
116,1937,Please send the new keyboard to my home addres...,Keyboard,Item sent,04-10-2022,04-10-2022,3,Shipping address for new keyboard.,"[-0.038060952, -0.099653475, 0.08719982, -0.06...",3,Device replacement and ordering accessories
